In [29]:
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt

In [30]:
train_data =  pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

# Filling Null Values

In [31]:
train_data.isnull().sum()[train_data.isnull().sum() > 0]
test_data.isnull().sum()[test_data.isnull().sum() > 0]

,0


# Encoding Data

In [32]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in train_data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))

    # Handle unseen labels in test set
    test_data[col] = test_data[col].astype(str)
    test_data[col] = test_data[col].apply(lambda x: x if x in le.classes_ else 'Unknown')

    # Add "Unknown" as a new possible class
    le.classes_ = np.append(le.classes_, 'Unknown')
    test_data[col] = le.transform(test_data[col])

    label_encoders[col] = le


parameter = [
    "road_signs_present","public_road","holiday","school_season"
]
for n in parameter:
  train_data[n] =  le.fit_transform(train_data[n])
  test_data[n] =  le.transform(test_data[n])

# Standadisation

In [33]:
from sklearn.preprocessing import StandardScaler

to_standardize = [
    "speed_limit"
]

scaler = StandardScaler()
train_data[to_standardize] = scaler.fit_transform(train_data[to_standardize])
test_data[to_standardize] = scaler.transform(test_data[to_standardize])

# Splitting the data

In [38]:
X_train = train_data.iloc[:,1:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,1:]

In [34]:
# train_data.to_csv("train_processed.csv", index=False)
# test_data.to_csv("test_processed.csv", index=False)

# Model Selection

In [39]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
parameters = {
    "n_estimators": [100, 200, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth": [3, 4, 5, 6],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "subsample": [0.6, 0.8, 1.0],
    "max_features": ["auto", "sqrt", "log2"]
}


In [40]:
regressor = GradientBoostingRegressor();
from sklearn.model_selection import RandomizedSearchCV

cv = RandomizedSearchCV(
    regressor,
    param_distributions=parameters,
    n_iter=30,
    cv=4,
    scoring='r2',
    random_state=42,
    n_jobs=-1,
    verbose=2
)


In [41]:
cv.fit(X_train,y_train)


Fitting 4 folds for each of 30 candidates, totalling 120 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
24 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/

{'subsample': 0.6, 'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.05}
0.8853097834849404


In [42]:
print(cv.best_params_)
print(cv.best_score_)

{'subsample': 0.6, 'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.05}
0.8853097834849404


In [43]:
y_pred = cv.predict(X_test)

In [45]:
import pandas as pd
import numpy as np

# Create submission DataFrame
submission = pd.DataFrame({
    "Id": range(517754, 690339),  # since end is exclusive
    "risk": y_pred
})

# Save to CSV
submission.to_csv("Playground.csv", index=False)
print("✅ File saved as submission_finale_xgboost_rf_ensemble.csv")

✅ File saved as submission_finale_xgboost_rf_ensemble.csv
